In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 6.0 MB/s eta 0:00:00


In [2]:
from typing import Dict, Text

import pandas as pd
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
df = pd.read_csv('/content/gdrive/MyDrive/ratings_Beauty.csv')

In [5]:
dfsubset = df.iloc[:100000]

In [6]:
dataset = tf.data.Dataset.from_tensor_slices(dict(dfsubset))

In [7]:
ratings = dataset.map(lambda x: {
    "ProductId": x["ProductId"],
    "UserId": x["UserId"]
})

products = dataset.map(lambda x: x["ProductId"])

In [8]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["UserId"]))

product_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
product_ids_vocabulary.adapt(products)

In [9]:
class BeautyRatingsModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      product_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.product_model = product_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["UserId"])
    product_embeddings = self.product_model(features["ProductId"])

    return self.task(user_embeddings, product_embeddings)

In [10]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
product_model = tf.keras.Sequential([
    product_ids_vocabulary,
    tf.keras.layers.Embedding(product_ids_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    products.batch(128).map(product_model)
  )
)

In [11]:
# Create a retrieval model.
model = BeautyRatingsModel(user_model, product_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    products.batch(100).map(lambda prod: (prod, model.product_model(prod))))

Epoch 1/3
25/25 [==============================] - 659s 26s/step - factorized_top_k/top_1_categorical_accuracy: 1.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 1.0000e-05 - factorized_top_k/top_10_categorical_accuracy: 1.0000e-05 - factorized_top_k/top_50_categorical_accuracy: 5.0000e-05 - factorized_top_k/top_100_categorical_accuracy: 1.0000e-04 - loss: 32419.0675 - regularization_loss: 0.0000e+00 - total_loss: 32419.0675
Epoch 2/3
25/25 [==============================] - 598s 24s/step - factorized_top_k/top_1_categorical_accuracy: 0.0015 - factorized_top_k/top_5_categorical_accuracy: 0.0015 - factorized_top_k/top_10_categorical_accuracy: 0.0015 - factorized_top_k/top_50_categorical_accuracy: 0.0018 - factorized_top_k/top_100_categorical_accuracy: 0.0029 - loss: 31824.7437 - regularization_loss: 0.0000e+00 - total_loss: 31824.7437
Epoch 3/3
25/25 [==============================] - 533s 21s/step - factorized_top_k/top_1_categorical_accuracy: 0.1977 - factorized_top_k/top_5_ca

In [12]:
# Get some recommendations.
_, prods = index(np.array(["A1Z513UWSAAO0F"]))
print(f"Top 10 recommendations for user A1Z513UWSAAO0F: {prods[0, :10]}")

Top 10 recommendations for user A1Z513UWSAAO0F: [b'0558925278' b'0558925278' b'B000052YPG' b'B000052YPG' b'B000052YPG'
 b'B000052YPG' b'B000052YPG' b'B000052YPG' b'B000052YPG' b'B000052YPG']
